# Simplicial cell complexes of $\mathbb{CP}^n$, $n=2,3,4$

This notebook contains code to produce our simplicial cell complex of $\mathbb{CP}^n$ for $n=2,3,4$ as a Regina triangulation, and for $n>4$, as a list of coloured edges (graph encoding). Note that support for $n$-manifolds in Regina is capped at real dimension $8$.

<i>Simplicial cell decompositions of $\mathbb{CP}^n$</i> by <a href="https://www.tcgcrest.org/people/basudeb-datta/">Basudeb Datta</a> and <a href="https://sites.google.com/view/jonathan-spreer/">Jonathan Spreer</a>, <a href="https://arxiv.org/abs/tba"> arXiv:tba </a>.


In [2]:
# Imports
import time, math, random, sys
import itertools

# import all functions etc. from regina
from regina import *

# helper functions in "./src/cpn.py"
sys.path.append("./src/")
from cpn import *

### Sphere products

Here we give code to produce our sphere products. These are simplicial cell complexes denoted by $X^n$ in the paper. The first procedure returns a regina triangulation. The second procedure returns a list of coloured edges.

In [3]:
def sphere_product(NN,triangulation,perm):
  '''
  return sphere product (works up to NN=4, regina only allows triangulations
  up to 8 = 2 x 4 real dimensions)

  NN            = (complex) dimension of complex projective space
  triangulation = appropriate regina function for generating a d-dimensional triangulation
  perm          = approoriate regina function for permuting elements of a simplex
                  (d+1) points
  '''
  O = orderings([[-1]*(2*NN)],1)
  O.sort()
  F = triangle_product(NN)
  faces,pos = boundary_faces(F)
  len_faces = len(F)
  Tri = simpToReg(F,2*NN,triangulation,perm)
  
  # for conversion to binary
  frmt = "0"+str(NN)+"b"
  
  # build up triangulation from disjoint triangulations of cells
  # cells are numbered by which number they represent in binary - 1
  T = triangulation()
  for c in range(2**NN):
    T.insertTriangulation(Tri)
  
  # loop over cells
  for c in range(2**NN):    
    # make buckets of free faces to connect to neighbour cells:
    buckets = []
    for i in range(NN):
      buckets.append([])
    for j in range(len(faces)):
      for char in range(NN-1):
        # n=2: len faces 18
        # 0 :
        # [0, 0, 9, 9, 0, 0, 0, 0, 0, 0]
        #
        # n=3: len faces 270
        # 0 :
        # [0, 0, 90, 180, 0, 0, 0, 0, 0, 0]
        # 1 :
        # [0, 0, 0, 0, 180, 90, 0, 0, 0, 0]
        #
        # n=4: len faces 7560
        # 0 :
        # [0, 0, 1890, 5670, 0, 0, 0, 0, 0, 0]
        # 1 :
        # [0, 0, 0, 0, 3780, 3780, 0, 0, 0, 0]
        # 2 :
        # [0, 0, 0, 0, 0, 0, 5670, 1890, 0, 0]
        #
        # Algo:
        # char=0: len=2 -> first
        #char=0, len=3 -> not first (done for n=2)
        #  char=1, len=4 -> second
        #  char=1, len=5 -> not first not second (done for n=3)
        #    char=2, len=6 -> third
        #    char=2, len=7 -> not first, second, third (done for n=4)
        #      ...
        # for entry 2*(char+1)
        ctr1 = 0
        # for entry 2*(char+1)+1
        ctr2 = 0
        res=[]
        for k in faces[j]:
          if not k%3**(char+1) in res:
            res.append(k%3**(char+1))
        if len(res) == 2*(char+1):
          buckets[char].append(pos[j])
          break
        if len(res) == 2*(char+1)+1 and char == NN-2:
          buckets[char+1].append(pos[j])

    # decide to which cell to glue to
    bnr = format(c, frmt)
    for char in range(NN):
      # already glued
      if bnr[char] == '1':
        continue
      ll = list(bnr)
      ll[char] = '1'
      newBnr=''.join(ll)
      newC = int(newBnr,2)
      for buc in buckets[char]:
        T.simplex(len_faces*c+buc[0]).join(buc[1],T.simplex(buc[0]+len_faces*newC),perm())
  return T

In [4]:
def sphere_product_gem(NN):
  '''
  return sphere product as a list of coloured edges (works for arbitrary values of NN)

  NN            = (complex) dimension of complex projective space
  '''
  O = orderings([[-1]*(2*NN)],1)
  O.sort()
  F = triangle_product(NN)
    
  # get dual edges of F
  base = faces_gem(F)
  for k in range(len(base)):
    base[k].sort()
    for e in base[k]:
      e.sort()
  # get boundary faces of F
  faces,pos = boundary_faces(F)
  len_faces = len(F)
  # for conversion to binary
  frmt = "0"+str(NN)+"b"
  # build up graph encoding from disjoint triangulations of cells
  # cells are numbered by which number they represent in binary - 1
  # T[i]: edges of colour i, 0 <= i <= 2*NN
  T = []
  for i in range(2*NN+1):
    T.append([])
  
  # loop over cells
  for c in range(2**NN):    
    # add all the dual edges from triangulations of cells
    for k in range(len(base)):
      for e in base[k]:
        T[k].append([e[0]+c*len_faces,e[1]+c*len_faces])    
    # make buckets of free faces to connect to neighbour cells:
    buckets = []
    for i in range(NN):
      buckets.append([])
    for j in range(len(faces)):
      for char in range(NN-1):
        # see routine 'sphere_product' above for details
        ctr1 = 0
        ctr2 = 0
        res=[]
        for k in faces[j]:
          if not k%3**(char+1) in res:
            res.append(k%3**(char+1))
        if len(res) == 2*(char+1):
          buckets[char].append(pos[j])
          break
        if len(res) == 2*(char+1)+1 and char == NN-2:
          buckets[char+1].append(pos[j])

    # decide to which cell to glue to
    bnr = format(c, frmt)
    for char in range(NN):
      # already glued
      if bnr[char] == '1':
        continue
      ll = list(bnr)
      ll[char] = '1'
      newBnr=''.join(ll)
      newC = int(newBnr,2)
      for buc in buckets[char]:
        dualedge = [len_faces*c+buc[0],buc[0]+len_faces*newC]
        T[buc[1]].append(dualedge)
    for k in range(len(T)):
      T[k].sort()
  return T


Test the code

In [5]:
sp = sphere_product(2,Triangulation4,Perm5)
sp_gem = sphere_product_gem(2)


print("Euler characteristic:",sp.eulerCharTri())
print("Face-vector:",sp.fVector())
print("Intersection form:",sp.intersectionForm())

print("\nSphere product as a gem")
for c in sp_gem:
    print(c)

print("\nSphere product as a Regina triangulation")
print(sp.detail())



Euler characteristic: 4
Face-vector: [9, 27, 58, 60, 24]
Intersection form: Even, rank = 2, sig = 0: [[ 2 -1 ] [ -1 0 ]]

Sphere product as a gem
[[0, 12], [1, 13], [2, 14], [3, 9], [4, 10], [5, 11], [6, 18], [7, 19], [8, 20], [15, 21], [16, 22], [17, 23]]
[[0, 12], [1, 3], [2, 4], [5, 11], [6, 18], [7, 9], [8, 10], [13, 15], [14, 16], [17, 23], [19, 21], [20, 22]]
[[0, 1], [2, 8], [3, 15], [4, 5], [6, 7], [9, 21], [10, 11], [12, 13], [14, 20], [16, 17], [18, 19], [22, 23]]
[[0, 6], [1, 2], [3, 4], [5, 17], [7, 8], [9, 10], [11, 23], [12, 18], [13, 14], [15, 16], [19, 20], [21, 22]]
[[0, 6], [1, 7], [2, 14], [3, 9], [4, 16], [5, 17], [8, 20], [10, 22], [11, 23], [12, 18], [13, 19], [15, 21]]

Sphere product as a Regina triangulation
Size of the skeleton:
  Pentachora: 24
  Tetrahedra: 60
  Triangles: 58
  Edges: 27
  Vertices: 9

Pentachoron gluing:
  Pent  |  glued to:     (0123)     (0124)     (0134)     (0234)     (1234)
  ------+-----------------------------------------------------

In [6]:
# Testing one dimension higher (6 real dimensions, S2 x S2 x S2)
sp_gem = sphere_product_gem(3)
for k in range(len(sp_gem)):
  print("\nColour", k)
  print(sp_gem[k])


Colour 0
[[0, 360], [1, 361], [2, 362], [3, 363], [4, 364], [5, 365], [6, 366], [7, 367], [8, 368], [9, 369], [10, 370], [11, 371], [12, 372], [13, 373], [14, 374], [15, 375], [16, 376], [17, 377], [18, 378], [19, 379], [20, 380], [21, 381], [22, 382], [23, 383], [24, 384], [25, 385], [26, 386], [27, 387], [28, 388], [29, 389], [30, 210], [31, 211], [32, 212], [33, 213], [34, 214], [35, 215], [36, 216], [37, 217], [38, 218], [39, 219], [40, 220], [41, 221], [42, 222], [43, 223], [44, 224], [45, 225], [46, 226], [47, 227], [48, 228], [49, 229], [50, 230], [51, 231], [52, 232], [53, 233], [54, 234], [55, 235], [56, 236], [57, 237], [58, 238], [59, 239], [60, 150], [61, 151], [62, 152], [63, 153], [64, 154], [65, 155], [66, 156], [67, 157], [68, 158], [69, 159], [70, 160], [71, 161], [72, 162], [73, 163], [74, 164], [75, 165], [76, 166], [77, 167], [78, 168], [79, 169], [80, 170], [81, 171], [82, 172], [83, 173], [84, 174], [85, 175], [86, 176], [87, 177], [88, 178], [89, 179], [90, 450]

### Complex Projective Spaces

Here we produce the quotient of our sphere product $X^n$ by the symmetric group action to obtain a simplicial cell complex of $\mathbb{CP}^n$.

In [71]:
def CP(NN,triangulation,perm):
  # calculate orbits
  orbs = orbits(NN)
  # compute sphere product
  sp = sphere_product(NN,triangulation,perm)
  # empty triangulation to contain complex projective space
  T = triangulation()
  # compute paths for simplicial subdivision of single cell
  paths = orderings([[-1]*(2*NN)],1)
  # sort (IMPORTANT to do this after every call of 'orderings(.,.)')
  paths.sort()
  # go through orbits and add one simplex per orbit
  for o in range(len(orbs)):
    T.newSimplex()
  # for every smallest orbit representative: detect gluing from sphere product sp
  for k in range(len(orbs)):
    # go through all orbit reps and check their adjacent orbits
    for simp in orbs[k]:
      for l in range(2*NN+1):
        adj = sp.simplex(simp).adjacentSimplex(l)
        target = False
        for o in range(len(orbs)):
          if adj.index() in orbs[o]:
            target = o
            break
    rep = orbs[k][0]
    for l in range(2*NN+1):
      adj = sp.simplex(rep).adjacentSimplex(l)
      # if face is still free
      if T.simplex(k).adjacentSimplex(l) == None:
        # get target simplex
        for o in range(len(orbs)):
          if adj.index() in orbs[o]:
            target = o
            break
        if T.simplex(target).adjacentSimplex(l) == None:
          T.simplex(k).join(l,T.simplex(target),perm())
  return T


In [74]:
def CP_gem(NN):
  # calculate orbits
  orbs = orbits(NN)
  # compute sphere product
  sp = sphere_product_gem(NN)
  # empty triangulation to contain complex projective space
  T = []
  for k in range(2*NN+1):
    T.append([])
  # compute paths for simplicial subdivision of single cell
  paths = orderings([[-1]*(2*NN)],1)
  # sort (IMPORTANT to do this after every call of 'orderings(.,.)')
  paths.sort()
  # for every smallest orbit representative: detect gluing from sphere product sp
  for k in range(len(orbs)):
    # go through all orbit reps and check their adjacent orbits
    rep = orbs[k][0]
    for l in range(2*NN+1):
      # find target simplex
      adj = None
      for e in sp[l]:
        if rep == e[0]:
          adj = e[1]
          break
        elif rep == e[1]:
          adj = e[0]
          break
      if adj == None:
        return None      
      # get target orbit
      for o in range(len(orbs)):
        if adj in orbs[o]:
          target = o
          break
      paired = False
      for e in T[l]:
        if target in e:
          paired = True
          break
      if paired:
        continue
      T[l].append([k,target])
  return T

Test the code

In [ ]:
cp2 = CP(2,Triangulation4,Perm5)
cp2_gem = CP_gem(2)

print("Euler characteristic:",cp2.eulerCharTri())
print("Face-vector:",cp2.fVector())
print("Intersection form:",cp2.intersectionForm(),"\n")

for k in range(len(cp2_gem)):
    print(cp2_gem[k])

In [99]:
# data for larger dimension
cp = CP_gem(3)
print("CP3 by colours")
for c in range(len(cp)):
    print("Colour",c)
    print(cp[c])

CP3 by colours
[[0, 57], [1, 58], [2, 59], [3, 54], [4, 55], [5, 56], [6, 47], [7, 46], [8, 45], [9, 52], [10, 53], [11, 49], [12, 48], [13, 51], [14, 50], [15, 87], [16, 88], [17, 89], [18, 90], [19, 91], [20, 92], [21, 78], [22, 79], [23, 80], [24, 75], [25, 76], [26, 77], [27, 83], [28, 84], [29, 81], [30, 82], [31, 86], [32, 85], [33, 99], [34, 100], [35, 101], [36, 95], [37, 96], [38, 93], [39, 94], [40, 98], [41, 97], [42, 104], [43, 103], [44, 102], [60, 105], [61, 106], [62, 107], [63, 108], [64, 109], [65, 110], [66, 111], [67, 112], [68, 113], [69, 114], [70, 115], [71, 116], [72, 117], [73, 118], [74, 119]] ,
[[0, 57], [1, 58], [2, 59], [3, 6], [4, 7], [5, 8], [9, 11], [10, 12], [13, 14], [15, 87], [16, 88], [17, 89], [18, 90], [19, 91], [20, 92], [21, 24], [22, 25], [23, 26], [27, 29], [28, 30], [31, 32], [33, 45], [34, 46], [35, 47], [36, 48], [37, 49], [38, 50], [39, 51], [40, 52], [41, 53], [42, 54], [43, 55], [44, 56], [60, 105], [61, 106], [62, 107], [63, 75], [64, 76]

In [100]:
# compare gem and regina triangulation

NN=4
cp_gem = CP_gem(NN)
cp = CP(NN,Triangulation8,Perm9)

for k in range(2*NN+1):
  chksm = []
  for n in range(cp.size()):
    chksm.append(0)
  err = False
  for n in range(cp.size()):
    adj = cp.simplex(n).adjacentSimplex(k)
    target = adj.index()
    e_reg = [n,target]
    e_reg.sort()
    if not e_reg in cp_gem[k]:
      err = True
    chksm[n]+=1
    chksm[target]+=1
  if max(chksm) == 2 and min(chksm) == 2 and not err:
    print("colour",k,"ok",err)
  else:
    print("\n\n\n\n\nERROR\n\n\n\n")

colour 0 ok False
colour 1 ok False
colour 2 ok False
colour 3 ok False
colour 4 ok False
colour 5 ok False
colour 6 ok False
colour 7 ok False
colour 8 ok False


### All examples

In [101]:
#sp2 = sphere_product(2,Triangulation4,Perm5)
#sp3 = sphere_product(3,Triangulation6,Perm7)
#sp4 = sphere_product(4,Triangulation8,Perm9)


#cp2 = CP(2,Triangulation4,Perm5)
#cp3 = CP(3,Triangulation6,Perm7)
cp4 = CP(4,Triangulation8,Perm9)

print("CP4:")
print("Euler characteristic:",cp4.eulerCharTri())
print("Face-vector:",cp4.fVector())

KeyboardInterrupt: 